# Spatial Data Assimilation in Geologic CO2 Sequestration
## Misael M. Morales - Summer 2023 - LANL

In [ ]:
# Load packages, class, and functions
from utils import *
sda = spatialDA()
sda.check_torch_gpu()
octave = sda.mrst_startup()

In [ ]:
# Load permeability (true, ensemble) and saturation (true)
perm_true, sat_true = sda.load_perm_sat_true()
perm_ens            = sda.load_perm_ens()
perm_all            = sda.load_perm_all()
sda.plot_perm_sat(perm_true, sat_true)

In [ ]:
# Reservoir Simulation of the permeability ensemble
n_realizations = 5#perm_ens.shape[0]

saturation = {}
for i in range(n_realizations):
    saturation[i] = octave.feval('simulations_octave/sim', perm_all[i])
    print('Simulation {} Done!'.format(i))

for i in range(2):
    sda.plot_perm_sat(perm_all[i], saturation[i])

***
### ESMDA-GEO

In [ ]:
def linear(p, x):
    return p[0]*np.exp(x*p[1])

def forward_model(m_ensemble, x):
    d_pred = np.zeros([m_ensemble.shape[0], x.shape[0]])
    for j in range(m_ensemble.shape[0]):
        d_pred[j, :] = linear(m_ensemble[j, :], x)
    return d_pred

a, b = 10.0, -0.002                                    # true parameters
x = np.arange(100)                                           # timesteps
obs = linear((a, b), x) + np.random.normal(0,1,x.shape[0])       # observations
cov_obs = np.diag([1.0] * obs.shape[0])                        # Observation error covariance matrix
print('obs: {} | cov_obs: {}'.format(obs.shape, cov_obs.shape))

n_ensemble = 100                                               # size of the ensemble
ma = np.random.uniform(low=-10.0, high=50.0, size=n_ensemble)  # Uniform law for the parameter a ensemble
mb = np.random.uniform(low=-0.001, high=0.01, size=n_ensemble) # Uniform law for the parameter b ensemble
m_ensemble = np.stack((ma, mb), axis=1)                        # Prior ensemble
print('m_ensemble: {}'.format(m_ensemble.shape))

n_assimilations = 3
# Use a geometric suite to compte alphas.
cov_obs_inflation_geo = 1.2
cov_obs_inflation_factors = [1.1]
for i in range(1, n_assimilations):
    cov_obs_inflation_factors.append(cov_obs_inflation_factors[i-1] / cov_obs_inflation_geo)
scaling_factor = np.sum(1 / np.array(cov_obs_inflation_factors))
cov_obs_inflation_factors = [alpha * scaling_factor for alpha in cov_obs_inflation_factors]
print('cov_obs_inflation_factors:', cov_obs_inflation_factors)
cov_mm_inflation_factors = [1.2] * n_assimilations       # This is just for the test
print('cov_mm_inflation_factors: ', cov_mm_inflation_factors)

solver = ESMDA(obs, m_ensemble, cov_obs, forward_model,
    forward_model_args        = (x,),
    forward_model_kwargs      = {},
    n_assimilations           = n_assimilations,
    cov_obs_inflation_factors = cov_obs_inflation_factors,
    cov_mm_inflation_factors  = cov_mm_inflation_factors,
    save_ensembles_history    = True)
solver.solve()

# Get the approximated parameters and uncertainty
a_approx, b_approx = np.average(solver.m_prior, axis=0)
a_std, b_std = np.sqrt(np.diagonal(solver.cov_mm))
# print(f"a = {a_approx:.5f} +/- {a_std:.4E}")
# print(f"b = {b_approx:.5f} +/- {b_std: 4E}")

posterior = np.zeros((n_ensemble, x.shape[0]))
for i in range(n_ensemble):
    posterior[i,:] = linear(solver.m_prior[i], np.arange(x.shape[0]))

prior = np.zeros((n_ensemble,x.shape[0]))
for i in range(n_ensemble):
    prior[i] = linear(m_ensemble[i], np.arange(x.shape[0]))
 
# plt.figure()
# for i in range(n_ensemble):
#     plt.plot(np.arange(x.shape[0]), prior[i], 'c-', alpha=0.2)
#     plt.plot(np.arange(x.shape[0]), obs, 'k-')
#     plt.plot(np.arange(x.shape[0]), posterior[i], 'r--')
# plt.show()

In [ ]:
approximate_cov_mm(m_ensemble)

In [ ]:
# Observations
obs     = np.array(list(saturation.values()))[:,0]
cov_obs = approximate_cov_mm(perm_ens[:n_realizations].T)
print('obs: {} | cov_obs: {}'.format(obs.shape, cov_obs.shape))

# Ensemble
n_ensemble = n_realizations
m_ensemble = perm_ens[:n_ensemble]
print('m_ensemble: {}'.format(m_ensemble.shape))

In [ ]:
n_assimilations = 2

cov_mm_inflation_geo  = 1.2
cov_obs_inflation_geo = 1.2

cov_obs_inflation_factors = [1.1]
for i in range(1, n_assimilations):
    cov_obs_inflation_factors.append(cov_obs_inflation_factors[i-1] / cov_obs_inflation_geo)
scaling_factor = np.sum(1/np.array(cov_obs_inflation_factors))
cov_obs_inflation_factors = [alpha * scaling_factor for alpha in cov_obs_inflation_factors]
print('cov_obs_inflation_factors:', cov_obs_inflation_factors)

cov_mm_inflation_factors = [cov_mm_inflation_geo] * n_assimilations
print('cov_mm_inflation_factors: ', cov_mm_inflation_factors)

In [ ]:
def forward_model(ens):
    d_pred = np.zeros([n_ensemble, sda.dim*sda.dim, 3])
    for i in range(ens.shape[0]):
        sat_pred = octave.feval('simulations_octave/sim', ens[i])
        d_pred[i,:,:] = np.moveaxis(sat_pred,0,-1)
    return d_pred

In [ ]:
solver = ESMDA(obs=obs, m_init=m_ensemble, cov_obs=cov_obs, 
               forward_model             = forward_model, 
               forward_model_args        = (),
               forward_model_kwargs      = {},
               n_assimilations           = n_assimilations,
               cov_obs_inflation_factors = cov_obs_inflation_factors,
               cov_mm_inflation_factors  = cov_mm_inflation_factors,
               save_ensembles_history    = True)
solver.solve()

***
# END